<a href="https://colab.research.google.com/github/pratikshaa12/NLP/blob/main/Spam%2Bdetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer


from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [4]:
import pandas as pd
import numpy as np

In [9]:
# Reading the given dataset
spam= pd.read_csv("/content/spam1.csv", encoding='latin-1', skipinitialspace=True, usecols=["v1", "v2"])

In [10]:
X= spam.v2
Y=spam.v1
le=LabelEncoder()
Y=le.fit_transform(Y)
print(Y)
y=Y.reshape(-1,1)
print(type(X))

[0 0 1 ... 0 0 0]
<class 'pandas.core.series.Series'>


In [11]:
X_train,X_test, Y_train,Y_test=train_test_split(X,Y,test_size=0.15)

In [12]:
X_train.shape

(4736,)

In [14]:
max_words=1000
max_len=150
tok=Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences=tok.texts_to_sequences(X_train)
print(sequences)
sequences_matrix=sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix


[[46, 37, 683], [13, 894, 31, 157, 9, 23, 282, 24, 345], [170, 259, 106, 44, 19, 128, 580, 450, 633, 61, 346, 70, 432, 31, 484, 27, 27, 658], [1, 433, 2, 59, 725, 7, 684, 12, 283], [68, 148, 110, 2, 94], [58, 4, 212, 62, 7, 1, 58, 215, 78, 372, 7, 20, 200, 259, 27, 63, 1, 38, 527, 1, 213, 434], [37, 503, 65], [1, 386, 581, 303, 31, 49, 83, 1, 107, 28, 227, 52, 58, 13, 414, 113, 6, 174, 10], [837, 4, 790, 237, 40, 559, 838, 44, 31, 49], [3, 140, 32, 137, 36, 3, 70, 470, 112, 1, 98, 54, 81], [839, 528, 471, 8, 42, 41], [560, 559, 15, 4, 293, 163, 118, 58, 11, 961, 11, 415, 248, 10, 11, 791, 387, 88, 23, 113, 11, 1, 215, 2, 138, 222, 11, 113, 24, 248], [52, 229, 529, 1, 238, 8, 5, 49, 256, 2, 962, 3, 24, 726], [582, 19, 95, 82, 527, 16, 20, 583, 34, 727, 322, 609, 530], [259, 402, 7], [30, 3, 685, 5], [28, 8, 26, 3, 70, 201], [102, 59, 2, 50, 266, 284], [1, 248, 22, 1, 58, 35, 3, 3, 561, 3, 86, 52, 451, 3, 21, 52, 128, 3, 634, 2, 10, 52, 128, 1, 74, 3, 8, 11, 168], [61, 4, 388, 15, 895], 

array([[  0,   0,   0, ...,  46,  37, 683],
       [  0,   0,   0, ..., 282,  24, 345],
       [  0,   0,   0, ...,  27,  27, 658],
       ...,
       [  0,   0,   0, ...,  41, 273, 104],
       [  0,   0,   0, ..., 787,  35,  10],
       [  0,   0,   0, ...,  12,  13,  95]], dtype=int32)

In [19]:
def RNN():
  inputs=Input(name='inputs', shape=[max_len])
  layer=Embedding(max_words,50,input_length=max_len)(inputs)
  layer=LSTM(64)(layer) ## long short term memory
  layer=Dense(256,name='FC1')(layer)  # Densely connected layer
  layer=Activation('relu')(layer)  # Activation function
  layer=Dropout(0.5)(layer)  #Improvisation of model using Dropouts
  layer=Dense(1,name='out_layer')(layer)  # final fully connected Dense layer
  layer=Activation('sigmoid')(layer)  #output layer
  model=Model(inputs=inputs,outputs=layer)
  return model

In [20]:
model=RNN()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)                  │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 150, 50)             │          50,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC1 (Dense)                          │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_layer (Dense)                    │ (None, 1)                   │             257 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 96,337 (376.32 KB)

 Trainable params: 96,337 (376.32 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(),metrics=['accuracy'])

In [23]:
model.fit(sequences_matrix,Y_train,batch_size=128, epochs=10, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 13s 317ms/step - accuracy: 0.8213 - loss: 0.4587 - val_accuracy: 0.9156 - val_loss: 0.2478
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 259ms/step - accuracy: 0.9372 - loss: 0.2007 - val_accuracy: 0.9599 - val_loss: 0.1396
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 12s 326ms/step - accuracy: 0.9776 - loss: 0.0895 - val_accuracy: 0.9726 - val_loss: 0.1048
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 11s 339ms/step - accuracy: 0.9764 - loss: 0.0782 - val_accuracy: 0.9800 - val_loss: 0.0653
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 283ms/step - accuracy: 0.9867 - loss: 0.0543 - val_accuracy: 0.9757 - val_loss: 0.0728
